1. Import Necessary Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

- `tensorflow` and `keras`: We use TensorFlow, a popular deep learning framework, and its Keras API for building and training neural networks.
- `ImageDataGenerator`: This utility helps in preprocessing and augmenting image data. It can also generate batches of image data with real-time data augmentation.
- `matplotlib.pyplot`: A library for creating visualizations, which we use to plot training and validation metrics.

2. Define the CNN Model

In [2]:
def create_model(num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(num_classes, activation='softmax')  # Multi-class classification
    ])
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',  # Categorical crossentropy for multi-class
                  metrics=['accuracy'])
    
    return model

- `create_model` function: This function builds a Convolutional Neural Network (CNN) for multi-class image classification.
- `Convolutional Layers (Conv2D)`: These layers detect features like edges, textures, and shapes in the images.
- `Max Pooling Layers (MaxPooling2D)`: These layers reduce the spatial dimensions (width and height) of the feature maps, reducing the number of parameters and computations.
- `Flatten Layer`: Converts the 2D matrix of features into a 1D vector, which can be fed into the Dense layers.
- `Dense Layers`: Fully connected layers that learn the final classification task.
- `softmax activation`: Used in the output layer for multi-class classification, converting logits to probabilities for each class.
- `compile method`: Configures the model for training. We use the Adam optimizer and categorical crossentropy loss, which are standard for multi-class classification tasks.

3. Create Data Generators

In [3]:
def create_data_generators(train_dir, validation_dir):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    validation_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical'  # Categorical for multi-class
    )

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical'  # Categorical for multi-class
    )
    
    return train_generator, validation_generator

- `create_data_generators function`: This function creates data generators for the training and validation datasets.
- `ImageDataGenerator`:
    - `rescale=1./255`: Normalizes the pixel values to the range [0, 1].
    - `Data Augmentation`: Applied to the training data to increase diversity (e.g., rotation, zoom, horizontal flips).
- `flow_from_directory`: This method loads images directly from the directory, automatically labeling them based on the folder structure. It generates batches of augmented data for training and validation.
- `target_size=(128, 128)`: Resizes all images to 128x128 pixels.
- `class_mode='categorical'`: Specifies that we are performing multi-class classification, and the labels will be one-hot encoded.

4. Create Test Data Generator

In [4]:
def create_test_generator(test_dir):
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical',  # Categorical for multi-class
        shuffle=False
    )
    return test_generator

- `create_test_generator function`: Similar to the training and validation data generators but used for the test set.
- `No Data Augmentation`: Decided not to apply augmentation to test data, as we want to evaluate the model on unaltered images.
- `shuffle=False`: Ensures that the order of images in the test set is preserved, which is essential for consistent evaluation.

5. Train the Model

In [5]:
def train_model(model, train_generator, validation_generator, epochs=30):
    history = model.fit(
        train_generator,
        steps_per_epoch=100,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=50
    )
    return history

- `train_model function`: Trains the CNN model using the training data, while validating on the validation data.
- `model.fit`: Runs the training loop for a specified number of epochs.
    - `steps_per_epoch=100`: Number of batches of images per epoch.
    - `validation_steps=50`: Number of batches of images to validate after each epoch.
- `history`: Stores the training process’s loss and accuracy metrics, which can be used later for analysis and visualization.

6. Plot Training Results

In [6]:
def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

- `plot_training_history function`: Visualizes the model’s performance over the training epochs.
- `Accuracy and Loss`: Plots both training and validation accuracy and loss to observe how the model learns over time.
- `plt.subplot`: Creates a 2x1 grid for plotting. The first plot shows accuracy, and the second shows loss.

7. Save the Model

In [7]:
def save_model(model, filename='plant_species_classifier.h5'):
    model.save(filename)

- `save_model function`: Saves the trained model to a file, allowing it to be loaded and used later without retraining

8. Evaluate the Model

In [8]:
def evaluate_model(model, test_generator):
    test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
    print(f'Test accuracy: {test_acc:.2f}')

- `evaluate_model function`: Evaluates the model on the test set.
- `steps calculation`: Ensures that all batches are covered during evaluation.
- `model.evaluate`: Returns the loss and accuracy on the test data, which are printed out.

9. Main Execution Block

In [9]:
if __name__ == "__main__":
    # Paths to your data
    train_dir = 'C:/Users/steve/AIALA/dataset/train/'
    validation_dir = 'C:/Users/steve/AIALA/dataset/validate/'
    test_dir = 'C:/Users/steve/AIALA/dataset/test/'
    
    # Create data generators
    train_generator, validation_generator = create_data_generators(train_dir, validation_dir)
    test_generator = create_test_generator(test_dir)
    
    print("Training classes:", train_generator.class_indices)
    print("Validation classes:", validation_generator.class_indices)
    print("Test classes:", test_generator.class_indices)
    
    # Get the number of classes
    num_classes = len(train_generator.class_indices)
    
    # Create and train the model
    model = create_model(num_classes)
    history = train_model(model, train_generator, validation_generator, epochs=30)
    
    # Plot the training history
    plot_training_history(history)
    
    # Save the model
    save_model(model, 'H5Data/plant_species_classifier.h5')
    
    # Evaluate the model on test data
    evaluate_model(model, test_generator)

Found 533 images belonging to 8 classes.
Found 113 images belonging to 8 classes.
Found 92 images belonging to 8 classes.
Training classes: {'Black_grass': 0, 'Charlock': 1, 'Cleavers': 2, 'Common_Chickweed': 3, 'Common_wheat': 4, 'Maize': 5, 'Small_flowered_Cranesbill': 6, 'Sugar_beet': 7}
Validation classes: {'Black_grass': 0, 'Charlock': 1, 'Cleavers': 2, 'Common_Chickweed': 3, 'Common_wheat': 4, 'Maize': 5, 'Small_flowered_Cranesbill': 6, 'Sugar_beet': 7}
Test classes: {'Black_grass': 0, 'Charlock': 1, 'Cleavers': 2, 'Common_Chickweed': 3, 'Common_wheat': 4, 'Maize': 5, 'Small_flowered_Cranesbill': 6, 'Sugar_beet': 7}


C:\Anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30


C:\Anaconda\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 17/100 ━━━━━━━━━━━━━━━━━━━━ 23s 289ms/step - accuracy: 0.1090 - loss: 2.0842

C:\Anaconda\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 66ms/step - accuracy: 0.1244 - loss: 2.0709 - val_accuracy: 0.1593 - val_loss: 2.0366
Epoch 2/30
  2/100 ━━━━━━━━━━━━━━━━━━━━ 21s 218ms/step - accuracy: 0.1406 - loss: 2.0231

KeyboardInterrupt: 

- `Directory Paths`: Specifies the locations of the training, validation, and test datasets.
- `Data Generators`: Calls functions to create the data generators.
- `Model Creation`: Builds the CNN model using the number of classes from the training data.
- `Training and Evaluation`: Trains the model, plots the training history, saves the model, and finally evaluates its performance on the test set.